In [2]:
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv

# Define the edge indices and node features for graph 1
g1_edge_index = torch.tensor([[0, 1], [1, 0]], dtype=torch.long)
g1_x = torch.tensor([[12], [23]], dtype=torch.float)
g1_y = ['kitchen', 'bathroom']

# Convert the NetworkX graph to a PyTorch geometric data object
g1_data = Data(x=g1_x, edge_index=g1_edge_index.t().contiguous(), y=None)

# Define the edge indices and node features for graph 2
g2_edge_index = torch.tensor([[0, 1], [1, 0]], dtype=torch.long)
g2_x = torch.tensor([[12], [23]], dtype=torch.float)
g2_y = ['bedroom', 'bathroom']

# Convert the NetworkX graph to a PyTorch geometric data object
g2_data = Data(x=g2_x, edge_index=g2_edge_index.t().contiguous(), y=None)

# Define the test data
test_edge_index = torch.tensor([[0, 1], [1, 0]], dtype=torch.long)
test_x = torch.tensor([[12], [23]], dtype=torch.float)
test_y = ['bedroom', 'kitchen']

# Convert the NetworkX graph to a PyTorch geometric data object
test_data = Data(x=test_x, edge_index=test_edge_index.t().contiguous(), y=None)

# Define a dictionary to map room names to integer labels
label_dict = {'kitchen': 0, 'bathroom': 1, 'bedroom': 2}

# Convert the room names to integer labels
g1_data.y = torch.tensor([label_dict[y] for y in g1_y], dtype=torch.long)
g2_data.y = torch.tensor([label_dict[y] for y in g2_y], dtype=torch.long)
test_data.y = torch.tensor([label_dict[y] for y in test_y], dtype=torch.long)

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(1, 16)
        self.conv2 = GCNConv(16, 3)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

def train(data):
    model.train()
    optimizer.zero_grad()
    loss = F.nll_loss(model(data.x.to(device), data.edge_index.to(device)), data.y.to(device))
    loss.backward()
    optimizer.step()
    return loss.item()

for epoch in range(200):
    loss1 = train(g1_data)
    loss2 = train(g2_data)
    print('Epoch: {:03d}, Graph 1 Loss: {:.5f}, Graph 2 Loss: {:.5f}'.format(epoch, loss1, loss2))
    
    
def test(data):
    model.eval()
    logits = model(data.x.to(device), data.edge_index.to(device))
    pred = logits.argmax(dim=1)
    test_loss = F.nll_loss(logits, data.y.to(device))
    test_acc = pred.eq(data.y.to(device)).sum().item() / len(data.y)
    return test_loss.item(), test_acc

test_loss, test_acc = test(test_data)
print('Test Loss: {:.5f}, Test Accuracy: {:.5f}'.format(test_loss, test_acc))

Epoch: 000, Graph 1 Loss: 6.53779, Graph 2 Loss: 22.88020
Epoch: 001, Graph 1 Loss: 9.35793, Graph 2 Loss: 10.16735
Epoch: 002, Graph 1 Loss: 3.59483, Graph 2 Loss: 14.60684
Epoch: 003, Graph 1 Loss: 0.74036, Graph 2 Loss: 6.35166
Epoch: 004, Graph 1 Loss: 2.84047, Graph 2 Loss: 7.02935
Epoch: 005, Graph 1 Loss: 1.31023, Graph 2 Loss: 0.80142
Epoch: 006, Graph 1 Loss: 2.33960, Graph 2 Loss: 6.88519
Epoch: 007, Graph 1 Loss: 1.20018, Graph 2 Loss: 3.97843
Epoch: 008, Graph 1 Loss: 0.74760, Graph 2 Loss: 4.28111
Epoch: 009, Graph 1 Loss: 1.49614, Graph 2 Loss: 5.80094
Epoch: 010, Graph 1 Loss: 2.36446, Graph 2 Loss: 3.01323
Epoch: 011, Graph 1 Loss: 2.70807, Graph 2 Loss: 5.11795
Epoch: 012, Graph 1 Loss: 1.17502, Graph 2 Loss: 1.57652
Epoch: 013, Graph 1 Loss: 3.92836, Graph 2 Loss: 1.28823
Epoch: 014, Graph 1 Loss: 2.29801, Graph 2 Loss: 2.79139
Epoch: 015, Graph 1 Loss: 2.06243, Graph 2 Loss: 1.73660
Epoch: 016, Graph 1 Loss: 6.57269, Graph 2 Loss: 2.34772
Epoch: 017, Graph 1 Loss: 2.

In [15]:
import pickle

with open('/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git/submission/ignore_outputs/data.pkl', 'rb') as f:
    embeddings, Y, df, X_all, edges_df_list, attributes_df_list = pickle.load(f)



In [24]:
import pickle

with open('/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git/submission/ignore_outputs/test_data.pkl', 'rb') as f:
    test_embeddings, test_Y, test_df, test_X_all, test_edges_df_list, test_attributes_df_list = pickle.load(f)



In [29]:
import pandas as pd
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch_geometric.data import DataLoader

room_classes_names = ["Background", "Outdoor", "Wall", "Kitchen", "Living Room" ,"Bed Room", "Bath", "Entry", "Railing", "Storage", "Garage"]

# Define a function to create a PyTorch Geometric Data object from the room and edge data for each floorplan
def create_data(attributes_df, edges_df):
    edge_index = edges_df[['source', 'target']].values.T
    x = attributes_df[['Area', 'Relative Area', 'Number of neighboring rooms']].values
    y = attributes_df['Room Type'].values
    return Data(x=torch.tensor(x, dtype=torch.float), edge_index=torch.tensor(edge_index, dtype=torch.long), y=torch.tensor(y, dtype=torch.long))

# Create a list of Data objects, one for each floorplan
data_list = []
for i in range(len(attributes_df_list)):
    attributes_df_i = attributes_df_list[i]
    edges_df_i = edges_df_list[i]
    data_list.append(create_data(attributes_df_i, edges_df_i))

# Define the GCN architecture
class Net(torch.nn.Module):
    def __init__(self, num_features, num_classes):
        super(Net, self).__init__()
        self.conv1 = GCNConv(num_features, 16)
        self.conv2 = GCNConv(16, num_classes)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Create the GCN and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(num_features=3, num_classes=len(room_classes_names)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

# Define the training loop
def train(model, optimizer, data_loader):
    model.train()
    total_loss = 0
    for data in data_loader:
        optimizer.zero_grad()
        out = model(data.x.to(device), data.edge_index.to(device))
        loss = F.nll_loss(out, data.y.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(data_loader.dataset)

# Train the GCN on your floorplan data
num_classes = len(room_classes_names)
model = Net(num_features=3, num_classes=len(room_classes_names)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
data_loader = DataLoader(data_list, batch_size=32, shuffle=True)
for epoch in range(200):
    loss = train(model, optimizer, data_loader)
    print('Epoch: {:03d}, Loss: {:.5f}'.format(epoch, loss))

# Define a function to create a PyTorch Geometric Data object from the room and edge data for each floorplan
def create_data(attributes_df, edges_df):
    edge_index = edges_df[['source', 'target']].values.T
    x = attributes_df[['Area', 'Relative Area', 'Number of neighboring rooms']].values
    return Data(x=torch.tensor(x, dtype=torch.float), edge_index=torch.tensor(edge_index, dtype=torch.long), y=None)

# Create a list of Data objects for the test floorplans
test_data_list = []
for i in range(len(test_attributes_df_list)):
    attributes_df_i = test_attributes_df_list[i].drop('Room Type', axis=1)
    edges_df_i = test_edges_df_list[i]
    test_data_list.append(create_data(attributes_df_i, edges_df_i))

# Use the trained model to predict room types for the test data
test_data_loader = DataLoader(test_data_list, batch_size=32, shuffle=False)
model.eval()
predictions = []
with torch.no_grad():
    for data in test_data_loader:
        logits = model(data.x.to(device), data.edge_index.to(device))
        pred = logits.argmax(dim=1)
        pred_room_types = [room_classes_names[i] for i in pred.tolist()]
        # Create a list of None values with the same length as the number of nodes in the graph
        pred_room_types = pred_room_types + [None]*(len(data.x) - len(pred_room_types))
        predictions.extend(pred_room_types)

# Create a list of actual room types for the test data
actual_room_types = []
for attributes_df_i in test_attributes_df_list:
    actual_room_types.extend(attributes_df_i['Room Type'].tolist())

# Compare the actual and predicted room types and calculate accuracy
correct_predictions = 0
for i in range(len(actual_room_types)):
    if actual_room_types[i] == predictions[i]:
        correct_predictions += 1
accuracy = correct_predictions / len(actual_room_types)

print('Accuracy: {:.2f}%'.format(accuracy * 100))



/Users/alishakhan/opt/miniconda3/envs/alishadev/lib/python3.9/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch: 000, Loss: 23515.63256
Epoch: 001, Loss: 21054.25994
Epoch: 002, Loss: 18218.35352
Epoch: 003, Loss: 11709.67205
Epoch: 004, Loss: 10716.26456
Epoch: 005, Loss: 9867.95055
Epoch: 006, Loss: 6936.04987
Epoch: 007, Loss: 6568.32253
Epoch: 008, Loss: 7471.30269
Epoch: 009, Loss: 5654.76778
Epoch: 010, Loss: 3556.99171
Epoch: 011, Loss: 3224.09651
Epoch: 012, Loss: 3891.99986
Epoch: 013, Loss: 3207.75101
Epoch: 014, Loss: 3070.81452
Epoch: 015, Loss: 2954.74326
Epoch: 016, Loss: 2626.25652
Epoch: 017, Loss: 2581.87993
Epoch: 018, Loss: 1944.55977
Epoch: 019, Loss: 1563.09123
Epoch: 020, Loss: 1880.79916
Epoch: 021, Loss: 1535.99977
Epoch: 022, Loss: 1441.22153
Epoch: 023, Loss: 1306.62601
Epoch: 024, Loss: 1042.31021
Epoch: 025, Loss: 928.41926
Epoch: 026, Loss: 706.12984
Epoch: 027, Loss: 753.73809
Epoch: 028, Loss: 868.83910
Epoch: 029, Loss: 545.39820
Epoch: 030, Loss: 541.76747
Epoch: 031, Loss: 503.56068
Epoch: 032, Loss: 549.88308
Epoch: 033, Loss: 338.69576
Epoch: 034, Loss: 

/Users/alishakhan/opt/miniconda3/envs/alishadev/lib/python3.9/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [33]:
from sklearn.metrics import classification_report, confusion_matrix

# Convert the room type labels from strings to integers
room_classes_int = {name: i for i, name in enumerate(room_classes_names)}
actual_room_types_int = [room_classes_int[r] for r in actual_room_types]
predicted_room_types_int = [room_classes_int[r] for r in predictions]

# Get the unique classes in the actual and predicted labels
unique_classes = list(set(actual_room_types_int + predicted_room_types_int))

# Convert the integer labels back to string labels
room_classes_names_int = {i: name for name, i in room_classes_int.items()}
target_names = [room_classes_names_int[i] for i in unique_classes]

# Calculate precision, recall, F1 score, and confusion matrix
print(classification_report(actual_room_types_int, predicted_room_types_int, labels=unique_classes, target_names=target_names))
print(confusion_matrix(actual_room_types_int, predicted_room_types_int, labels=unique_classes))


              precision    recall  f1-score   support

     Kitchen       0.00      0.00      0.00       137
 Living Room       0.27      0.18      0.21       131
    Bed Room       0.00      0.00      0.00       217
        Bath       0.44      0.15      0.23       291
       Entry       0.17      0.80      0.28       174
     Storage       0.00      0.00      0.00        68
      Garage       0.00      0.00      0.00         2

    accuracy                           0.20      1020
   macro avg       0.13      0.16      0.10      1020
weighted avg       0.19      0.20      0.14      1020

[[  0  16   0   6 115   0   0]
 [  0  23   0   3 105   0   0]
 [  0  17   0  13 187   0   0]
 [  0  16   0  45 230   0   0]
 [  0  10   0  24 140   0   0]
 [  0   2   0  11  55   0   0]
 [  0   0   0   0   2   0   0]]


/Users/alishakhan/opt/miniconda3/envs/alishadev/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/alishakhan/opt/miniconda3/envs/alishadev/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/alishakhan/opt/miniconda3/envs/alishadev/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf